In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module): #nn.Module을 상속받음
    def __init__(self):
        super(Net, self).__init__()
        #fc : fully connected
        self.fc1 = nn.Linear(16*5*5, 120) #nn.Linear(input, output) 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [ ]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [ ]:
input = torch.randn(1, 1, 32, 32)
net.zero_grad()
out = net(input)
out.backward(torch.randn(1, 10))

output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()
loss = criterion(output, target)

In [ ]:
net.zero_grad()   

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0040, -0.0034, -0.0022,  0.0058,  0.0144,  0.0019])


In [ ]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [ ]:
f.data

tensor([ 0.0191,  0.0341,  0.0139,  0.0930, -0.0554, -0.0239,  0.0062,  0.0924,
        -0.0850, -0.0111])

In [ ]:
import torch.optim as optim

# Optimizer를 생성합니다.
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 학습 과정(training loop)은 다음과 같습니다:
optimizer.zero_grad()   # 변화도 버퍼를 0으로 ( 그때마다 0으로 해줘야됨)
output = net(input)
loss = criterion(output, target)  #criterion : 앞에서 만든 손실함수
loss.backward()
optimizer.step()    # 업데이트 진행